# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [16]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date
import json
from dotenv import load_dotenv
import os

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [ ]:
### 💡 Première requête guidée
def get_weather_data(city):
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    load_dotenv()
    api_key = os.getenv("API_KEY_TEST")
    BASE_URL = "http://api.openweathermap.org/data/2.5"
    # URL : current weather data
    url = f"{BASE_URL}/weather"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # Celsius
        'lang': 'en'
    }

    # Votre code ici pour l'appel API
    # Gérez les erreurs HTTP !
    try :
        # send a get request to the API with the url and the parameters 
        responde = requests.get(url, params=params)
        
        # raise an HTTP error if responde contains an error status code 
        responde.raise_for_status()
        
        # return the JSON content of the responde 
        return responde.json()
        
    #
    except requests.exceptions.HTTPError as http_err:
        
        #if error code is 404 , the city is not found 
        if http_err == 404 :
            
            print(f"The City {city} is not found!")
        else : 
            
            #for other http errors print full error msg 
            print(f"HTTP Error is occurred : {http_err} ")
            
    except Exception as err:
        
        #catch any unexpected errors 
        print('Unexpected Error :{err}')
        
    return None

In [11]:
city_weather = get_weather_data("Rome, Lazio")
print(city_weather)

{'coord': {'lon': -85.1647, 'lat': 34.257}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 23.22, 'feels_like': 24.08, 'temp_min': 22.36, 'temp_max': 23.98, 'pressure': 1017, 'humidity': 95, 'sea_level': 1017, 'grnd_level': 990}, 'visibility': 10000, 'wind': {'speed': 0, 'deg': 0}, 'clouds': {'all': 0}, 'dt': 1753782831, 'sys': {'type': 2, 'id': 2009938, 'country': 'US', 'sunrise': 1753786170, 'sunset': 1753836272}, 'timezone': -14400, 'id': 4219762, 'name': 'Rome', 'cod': 200}


In [ ]:
# Configuration API
API_KEY = os.getenv("API_KEY_TEST")  # Remplacez par votre vraie clé
BASE_URL = "http://api.openweathermap.org/data/2.5"

# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Rome, Lazio", "Amsterdam", "Vienna"]

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [ ]:
# Configuration Visual Crossing
VC_API_KEY = os.getenv("API_KEY2")
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date, api_key):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

    url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"
    params = {
        'key': api_key,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
    }
    try :
        respo = requests.get(url, params=params)
        respo.raise_for_status()
        historical_data =  respo.json()
        return historical_data

    except requests.exceptions.HTTPError as http_err :
        if respo.raise_for_status == 404 :
         print(f"the city {city} is not found!")
        else :
            print(f"HTTP error is occurred : {http_err}")
    except Exception as err :
        print("unexpected error {err}")
        return None

In [ ]:
today = date.today()
# print(type(today))
last_week = today - timedelta(days=7)
print(last_week)
histo_data_json = get_historical_weather("Paris", last_week, today, VC_API_KEY)

2025-07-22


In [29]:
historical_weather = []
heart_json = histo_data_json['days']

for day in heart_json :
    historical_weather.append({
            "date": day["datetime"],           # Date of the record
            "temperature": day["temp"],        # Average temperature
            "humidity": day["humidity"],       # Humidity percentage
            "precipitation": day["precip"],    # Precipitation amount
            "wind_speed": day["windspeed"],    # Wind speed
            "pressure": day["pressure"]        # Atmospheric pressure
    })
    
    df_historical_weather = pd.DataFrame(historical_weather)


In [30]:
df_historical_weather.head(10)

,date,temperature,humidity,precipitation,wind_speed,pressure
0,2025-07-22,66.6,71.0,0.002,10.4,1013.4
1,2025-07-23,66.6,74.3,0.079,8.4,1013.8
2,2025-07-24,65.2,83.8,0.489,9.6,1014.9
3,2025-07-25,69.3,72.8,0.000,7.6,1017.7
4,2025-07-26,70.6,67.6,0.000,10.0,1018.2
5,2025-07-27,65.9,78.8,0.221,14.4,1016.4
6,2025-07-28,66.0,65.9,0.000,8.6,1019.9
7,2025-07-29,63.9,73.1,0.020,9.2,1020.4


### 🎯 Mission pratique
Récupérez les données des 7 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [69]:

# Load API key
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Step 1: Get coordinates
def get_coordinates(city_name):
    url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {"q": city_name, "limit": 1, "appid": API_KEY}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0]["lat"], data[0]["lon"]
        else:
            print(f"[!] No coordinates found for {city_name}")
            return None, None
    except Exception as e:
        print(f"[ERROR] Failed to get coordinates for {city_name}: {e}")
        return None, None

# Step 2: Get air quality
def get_air_quality(city, lat, lon):
    BASE_URL = "http://api.openweathermap.org/data/2.5/air_pollution"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": API_KEY
    }

    try:
        resp = requests.get(BASE_URL, params=params)
        resp.raise_for_status()
        air_q = resp.json()

        # Extract useful info
        air_info = []
        for i in air_q.get("list", []):
            aqi = i.get("main", {}).get("aqi")
            components = i.get("components", {})
            entry = {
                "city": city,
                "aqi": aqi,
                **components
            }
            air_info.append(entry)

        df = pd.DataFrame(air_info)
        print(df)
        return df

    except requests.exceptions.HTTPError as http_err:
        if resp.status_code == 404:
            print(f"City not found for coordinates ({lat}, {lon})!")
        else:
            print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Unexpected error: {err}")

# Example usage
city = "Paris"
lat, lon = get_coordinates(city)
if lat and lon:
    get_air_quality(city, lat, lon)


    city  aqi      co    no   no2     o3   so2  pm2_5  pm10   nh3
0  Paris    2  115.82  0.07  0.68  75.91  0.13   1.12  1.43  0.98


In [62]:
        # cities_a_q = []
        # for city in CITIES :
        #     if lat and lon in air_q :
        #         info = air_q["list"][0]
        #         components = info["components"]
        #         cities_a_q.append({
        #         "city": city,
        #         "lat": lat,
        #         "lon": lon,
        #         "aqi": info["main"]["aqi"],
        #         "co": components.get("co"),
        #         "no": components.get("no"),
        #         "no2": components.get("no2"),
        #         "o3": components.get("o3"),
        #         "so2": components.get("so2"),
        #         "pm2_5": components.get("pm2_5"),
        #         "pm10": components.get("pm10"),
        #         "nh3": components.get("nh3")
        #         })
        #     else:
        #         print(f"[!] No AQI data for {city}")

In [ ]:
### 🗺️ Obtenir les coordonnées
def get_city_coordinates(city, api_key):
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """
    pass



## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [11]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [ ]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [12]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites